## Install libraries for working with large language models (LLMs).

In [ ]:
!pip install langchain langchain_groq langchain_community

## Retrieve Groq API key from Colab environment

In [ ]:
from google.colab import userdata
groq_api_key=userdata.get('Groq')
# print(groq_api_key)

## Initialize a ChatGroq instance using the provided Groq API key and create an LLM object for interaction

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama3-groq-70b-8192-tool-use-preview")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78580a1134f0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78580a110a90>, model_name='llama3-groq-70b-8192-tool-use-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
llm.invoke('Hi, how r u?')

AIMessage(content="I'm doing well, thanks! How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 16, 'total_tokens': 31, 'completion_time': 0.04582809, 'prompt_time': 0.004788276, 'queue_time': 0.028527623000000002, 'total_time': 0.050616366}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'stop', 'logprobs': None}, id='run-da8a0156-41c4-487a-ac58-d29e2d54d63c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 15, 'total_tokens': 31})

## Define the State for the Chatbot

In [ ]:
from typing import Annotated  # Import the Annotated class for type hints with additional metadata
from typing_extensions import TypedDict  # Import the TypedDict class for defining custom typed dictionaries

from langgraph.graph import StateGraph  # Import the StateGraph class for creating state graphs
from langgraph.graph.message import add_messages  # Import the add_messages function for adding messages to a list

## Define a typed dictionary to represent the state of the chatbot

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]  # List of messages, with add_messages function applied

## Create a StateGraph instance, specifying the State type

In [ ]:
graph_builder = StateGraph(State)

# Define a chatbot function that takes a state as input and returns a new state with updated messages
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}  # Invoke the LLM on the input messages

# Add the chatbot node to the graph, specifying its name and the chatbot function
graph_builder.add_node("chatbot", chatbot)

# Set the chatbot node as both the entry point and finish point of the graph
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")

# Compile the graph into a runnable representation
graph = graph_builder.compile()

## Stream Graph Updates Based on User Input

In [ ]:
def stream_graph_updates(user_input: str):
    """
    Streams updates from the compiled graph based on user input.

    Args:
        user_input (str): The user's input message.

    Prints:
        str: The assistant's response message.
    """

    # Create an initial state with the user's input message
    initial_state = {"messages": [("user", user_input)]}

    # Stream updates from the compiled graph
    for event in graph.stream(initial_state):
        # Loop through each event's values (should be only one in this case)
        for value in event.values():
            # Print the assistant's latest response message
            print("Assistant:", value["messages"][-1].content)

In [ ]:
while True:
    try:
        user_input = input("User: ")  # Get user input

        if user_input.lower() in ["quit", "exit", "q"]:  # Check for exit commands
            print("Goodbye!")
            break  # Exit the loop

        stream_graph_updates(user_input)  # Process the user input using the graph
    except Exception as e:  # Catch any exceptions that might occur during input or processing
        print('Exceptions occured during processing:', e)
        stream_graph_updates(user_input)  # Process the default input
        break  # Exit the loop after the fallback

User: Hi?
Assistant: Hi! How can I assist you today?
User: can you give me the history of united states of america?
Assistant: The history of the United States of America is a long and complex one, spanning thousands of years. It begins with the pre-Columbian era, during which indigenous peoples inhabited the region that is now the United States. Following the arrival of European explorers in the late 15th century, the area was colonized by various European powers, including Spain, France, the Netherlands, and England. In 1776, the Thirteen American Colonies declared their independence from Great Britain, marking the beginning of the United States as a sovereign nation.

The early years of the United States were marked by westward expansion, conflicts with Native American tribes, and the institution of slavery. The Civil War, fought between the Union (the northern states) and the Confederacy (the southern states) from 1861 to 1865, resulted in the abolition of slavery. The late 19th an

KeyboardInterrupt: Interrupted by user